# How does targeted cash assistance affect incentives to work?


In [1]:
try:
    import openfisca_us
except ImportError as e:
    !pip install openfisca-us==0.73.0
    !mkdir output

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from openfisca_us import IndividualSim, Microsimulation
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables


In [3]:
# Set numbers from proposal
POV_THRESHOLD = 2
MONTHLY_AMOUNT = 500
FULL_DURATION_MONTHS = 18


class cambridge_cash_18mo(Variable):
    value_type = float
    entity = SPMUnit
    definition_period = YEAR

    def formula(spm_unit, period, parameters):
        pov_ratio = spm_unit("school_meal_fpg_ratio", period)
        income_eligible = pov_ratio < POV_THRESHOLD
        has_children = add(spm_unit, period, ["is_child"]) > 0
        return (
            income_eligible
            * has_children
            * MONTHLY_AMOUNT
            * FULL_DURATION_MONTHS
        )


class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
    def formula(spm_unit, period, parameters):
        original_net_income = baseline_variables[
            "spm_unit_net_income"
        ].formula(spm_unit, period, parameters)
        return original_net_income + spm_unit("cambridge_cash_18mo", period)


class add_cambridge_cash_18mo(Reform):
    def apply(self):
        self.update_variable(cambridge_cash_18mo)
        self.update_variable(spm_unit_net_income)


Run IndividualSim.


In [4]:
def family_sim(reform=None):
    if reform is None:
        sim = IndividualSim(year=2022)
    else:
        sim = IndividualSim(reform, year=2022)
    sim.add_person(name="p1", age=30)
    sim.add_person(name="p2", age=30)
    sim.add_person(name="c1", age=10)
    sim.add_person(name="c2", age=10)
    MEMBERS = ["p1", "p2", "c1", "c2"]
    # Exclude ACA for now.
    sim.add_tax_unit(members=MEMBERS, premium_tax_credit=0)
    sim.add_spm_unit(
        name="spm_unit",
        members=MEMBERS,
        broadband_cost=600,
        housing_cost=24_000,
    )
    sim.add_household(name="household", members=MEMBERS, state_code_str="MA")
    sim.vary("employment_income", max=100_000)
    return sim


baseline = family_sim()
reformed = family_sim(add_cambridge_cash_18mo)


Marginal tax rates


In [5]:
def make_df(sim, scenario):
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            net_income=sim.calc("spm_unit_net_income")[0].round(0),
            mtr=1
            - sim.deriv(
                "spm_unit_net_income", "employment_income", wrt_target="p1"
            ),
            scenario=scenario,
        )
    )


df = pd.concat(
    [make_df(baseline, "Baseline"), make_df(reformed, "Cambridge cash")]
)


In [6]:
def gap(sim):
    employment_income = sim.calc("employment_income")[0]
    net_income = sim.calc("spm_unit_net_income")[0]
    diffs = np.diff(net_income, append=np.inf)
    cliffs = np.where(diffs < 0)[0]
    l = []
    for cliff in cliffs:
        employment_income_before_cliff = employment_income[cliff]
        net_income_before_cliff = net_income[cliff]
        ix_first_exceed_cliff = np.argmax(net_income > net_income_before_cliff)
        employment_income_after_cliff = employment_income[
            ix_first_exceed_cliff
        ]
        l += [[employment_income_before_cliff, employment_income_after_cliff]]
    return l


In [7]:
def shade_cliffs(
    cliffs: list,
    fig: go.Figure,
    fillcolor: str,
    ymax: float,
) -> None:
    """Shades the cliffs in a net income or MTR chart.
    :param sim: Simulation.
    :type baseline: IndividualSim
    :param config: Configuration.
    :type config: Type[PolicyEngineResultsConfig]
    :param fig: Plotly figure.
    :type fig: go.Figure
    :param fillcolor: Fill color.
    :type fillcolor: str
    :param ymax: Maximum y value.
    :type ymax: float
    :return: None
    :rtype: None
    """
    for cliff in cliffs:
        start = cliff[0]
        end = cliff[1]
        if end > start:
            text = "This household is worse off earning between ${:,}".format(
                int(start)
            ) + " and ${:,}".format(int(end))
            fig.add_trace(
                go.Scatter(
                    x=[start, start, end, end, start],
                    y=[0, ymax, ymax, 0, 0],
                    fill="toself",
                    mode="lines",
                    fillcolor=fillcolor,
                    name="",
                    text=text,
                    opacity=0.3,
                    line_width=0,
                    showlegend=False,
                )
            )


In [8]:
from plotly import graph_objects as go

LABELS = dict(
    employment_income="Employment income",
    net_income="Net income",
    mtr="Marginal tax rate",
    scenario="Scenario",
)

COLORS = {"Baseline": "#606060", "Cambridge cash": "#1a77d2"}
LIGHT_COLORS = {"Baseline": "#c2c2c2", "Cambridge cash": "#65b6f6"}

fig = go.Figure()

line_chart = px.line(
    df[df.scenario == "Baseline"],
    "employment_income",
    "net_income",
    labels=LABELS,
    color_discrete_map=COLORS,
)


baseline_gap_bounds = gap(baseline)

ymax = df[df.scenario == "Baseline"].net_income.max()

shade_cliffs(baseline_gap_bounds, fig, LIGHT_COLORS["Baseline"], ymax)

fig.add_traces(list(line_chart.select_traces()))
fig.update_traces(line_color=COLORS["Baseline"])

fig.update_layout(
    xaxis_title="Employment income",
    yaxis_title="Net income",
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    showlegend=False,
)


def show_export(fig, name):
    fig.update_layout(plot_bgcolor="white")
    fig.show()
    fig.write_html("output/" + name + ".html")
    fig.write_image("output/" + name + ".png")


show_export(fig, "baseline")


Now add child allowance.


In [9]:
line_chart = px.line(
    df,
    "employment_income",
    "net_income",
    color="scenario",
    title="Family with two parents and two children",
    labels=LABELS,
    color_discrete_map=COLORS,
)

reformed_gap_bounds = gap(reformed)

fig = go.Figure()

shade_cliffs(baseline_gap_bounds, fig, LIGHT_COLORS["Baseline"], ymax)
shade_cliffs(reformed_gap_bounds, fig, LIGHT_COLORS["Cambridge cash"], ymax)

fig.update_layout(
    xaxis_title="Employment income",
    yaxis_title="Net income",
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
)
fig.add_traces(list(line_chart.select_traces()))

show_export(fig, "cambridge-cash")


In [10]:
baseline_gap_bounds


[[55500.0, 69500.0]]

Microsim: 2020 is latest year.


In [11]:
# Redefine for 12 instead of 18 months.
class cambridge_cash_12mo(Variable):
    value_type = float
    entity = SPMUnit
    definition_period = YEAR

    def formula(spm_unit, period, parameters):
        pov_ratio = spm_unit("school_meal_fpg_ratio", period)
        income_eligible = pov_ratio < POV_THRESHOLD
        has_children = add(spm_unit, period, ["is_child"]) > 0
        return income_eligible * has_children * MONTHLY_AMOUNT * 12


class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
    def formula(spm_unit, period, parameters):
        original_net_income = baseline_variables[
            "spm_unit_net_income"
        ].formula(spm_unit, period, parameters)
        # vehicle_payment = add(spm_unit, period, ["per_vehicle_payment"])
        return original_net_income + spm_unit("cambridge_cash_12mo", period)


class add_cambridge_cash_12mo(Reform):
    def apply(self):
        self.update_variable(cambridge_cash_12mo)
        self.update_variable(spm_unit_net_income)


In [12]:
mbaseline = Microsimulation(year=2020)
mreformed = Microsimulation(add_cambridge_cash_12mo, year=2020)

# Filter to MA.
person_weights = mbaseline.calc("person_weight")
spm_unit_weights = mbaseline.calc("spm_unit_weight")
household_weights = mbaseline.calc("household_weight")
state_code = mbaseline.calc("state_code_str", map_to="person")
state_code_household = mbaseline.calc("state_code_str")

for m in [mbaseline, mreformed]:
    m.set_input("person_weight", m.year, person_weights * (state_code == "MA"))
    m.set_input(
        "household_weight",
        m.year,
        household_weights * (state_code_household == "MA"),
    )


def get_metrics(sim, name):
    pov = sim.calc("spm_unit_is_in_spm_poverty", map_to="person")
    deep_pov = sim.calc("spm_unit_is_in_deep_spm_poverty", map_to="person")
    child = sim.calc("is_child")
    return pd.Series(
        dict(
            poverty=pov.mean(),
            child_poverty=(pov * child).sum() / child.sum(),
            deep_poverty=deep_pov.mean(),
            deep_child_poverty=(deep_pov * child).sum() / child.sum(),
        ),
        name=name,
    )


poverty_impact = pd.DataFrame(
    [
        get_metrics(mbaseline, "Baseline"),
        get_metrics(mreformed, "Cambridge cash"),
    ]
)
poverty_impact.loc["Diff"] = (
    poverty_impact.loc["Cambridge cash"] / poverty_impact.loc["Baseline"] - 1
)
poverty_impact


,poverty,child_poverty,deep_poverty,deep_child_poverty
Baseline,0.080950,0.081025,0.023105,0.012157
Cambridge cash,0.069169,0.047354,0.019330,0.000000
Diff,-0.145540,-0.415559,-0.163392,-1.000000


## Child allowance comparison

Start with total cost of Cambridge cash assistance if rolled out across MA, and number of children in MA.


In [13]:
budget = mreformed.calc("cambridge_cash_12mo", map_to="household").sum()
budget / 1e9


1.1851755007324218

In [14]:
kids = mreformed.calc("is_child").sum()
kids / 1e6


1.350342938720703

In [15]:
budget_neutral_monthly_child_allowance_amount = (budget / kids) / 12
budget_neutral_monthly_child_allowance_amount


73.14040176683076

In [16]:
def make_child_allowance(amount):
    class child_allowance(Variable):
        value_type = float
        entity = Person
        definition_period = YEAR

        def formula(person, period, parameters):
            return amount * 12 * person("is_child", period)

    class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
        def formula(spm_unit, period, parameters):
            original_net_income = baseline_variables[
                "spm_unit_net_income"
            ].formula(spm_unit, period, parameters)
            return original_net_income + add(
                spm_unit, period, ["child_allowance"]
            )

    class add_child_allowance(Reform):
        def apply(self):
            self.update_variable(child_allowance)
            self.update_variable(spm_unit_net_income)

    m = Microsimulation(add_child_allowance, year=2020)
    m.set_input("person_weight", m.year, person_weights * (state_code == "MA"))
    m.set_input(
        "household_weight",
        m.year,
        household_weights * (state_code_household == "MA"),
    )

    res = get_metrics(m, str(amount))
    res["cost"] = m.calc("child_allowance", map_to="household").sum()
    res["monthly_child_allowance"] = amount
    return pd.DataFrame(res).T


In [17]:
l = []
for i in range(0, 200, 15):
    l.append(make_child_allowance(i))

ca_df = pd.concat(l)


In [18]:
POVERTY_METRICS = [
    "poverty",
    "child_poverty",
    "deep_poverty",
    "deep_child_poverty",
]
ca_df[[i + "_chg" for i in POVERTY_METRICS]] = (
    ca_df[POVERTY_METRICS] / ca_df[POVERTY_METRICS].iloc[0] - 1
)


In [19]:
LABELS = dict(
    monthly_child_allowance="Monthly child allowance",
    cost="Cost",
    cost_rounded="Cost",
    poverty="Poverty rate",
    child_poverty="Child poverty rate",
    deep_poverty="Deep poverty rate",
    deep_child_poverty="Deep child poverty rate",
    poverty_chg="Poverty rate change",
    child_poverty_chg="Child poverty rate change",
    deep_poverty_chg="Deep poverty rate change",
    deep_child_poverty_chg="Deep child poverty rate change",
)
ca_df["child_allowance_print"] = ca_df.apply(
    lambda x: "${:,.0f} monthly child allowance".format(
        x.monthly_child_allowance
    ),
    axis=1,
)
ca_df["cost_rounded"] = ca_df["cost"].apply(lambda x: round(x, -7))
fig = px.line(
    ca_df,
    x="cost_rounded",
    y="poverty_chg",
    hover_name="child_allowance_print",
    hover_data=dict(
        child_poverty_chg=":.0%",
        deep_poverty_chg=":.0%",
        deep_child_poverty_chg=":.0%",
    ),
    labels=LABELS,
)
fig.add_hline(
    poverty_impact.loc["Diff"].poverty,
    line_color="gray",
    line_dash="dash",
    annotation_text="Cambridge cash",
    annotation_position="top left",
)
fig.add_vline(budget, line_color="gray", line_dash="dash")
# add_vline doesn't support rotation, so add annotation manually.
fig.add_annotation(
    x=budget + 5e7,
    y=-0.1,
    text="Cambridge cash",
    showarrow=False,
    textangle=90,
)
# Add a zero line.
fig.add_hline(0, line_color="gray", line_width=1)
fig.update_layout(
    xaxis_tickprefix="$",
    yaxis_tickformat=".0%",
)

show_export(fig, "child-allowance")
